# Facenet recreation
Google facenet recreation

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import itertools
from matplotlib import image
import glob as glob
from PIL import Image

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print("Using the GPU!")
else:
  print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")


# Build Dataset from facenet
using instructions from pytorch docs: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
Reading pairs to ensure that muiltiple faces of a single person 


In [ ]:
class LabledFacesWild(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        label_dict = []
        with open(annotations_file) as f:
            for line in f:
                name, i1, i2 = line.split('\t')
                label_dict.append({
                    "label": name,
                    "img1": f"{name}/{i1:04d}.jpg",
                    "img2": f"{name}/{i2:04d}.jpg"
                })
        self.img_labels = pd.DataFrame.from_records(label_dict)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels) * 2

    def __getitem__(self, idx):
        pair_num = idx // 2
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[pair_num, (idx % 2) + 1])
        image = read_image(img_path)
        label = self.img_labels.iloc[pair_num, 0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# actually load data
num_train_pairs = 1100
num_test_pairs = 500
batch_size = 40

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# construct the dataloader
train_data = Edges2Image('./pairsDevTrain.txt', './lfw/', transform)
test_data = Edges2Image('./pairsDevTest.txt', './lfw/', transform)
train_loader = DataLoader(train_data, batch_size=50, shuffle=False)
test_loader = DataLoader(test_data, batch_size=50, shuffle=False)

# todo figure out actual batching while including groups

# Create Facenet Model
Facenet model https://arxiv.org/pdf/1503.03832.pdf
Google Lenet 22layer: https://arxiv.org/pdf/1409.4842.pdf
input size = 250x250x3
use relu


In [ ]:
class Facenet_NN1(nn.Module):
    def __init__(self):
        super().__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        # torch.nn.LocalResponseNorm(size, alpha=0.0001, beta=0.75, k=1.0)
        self.conv1 = nn.Conv2d(3, 64, 7, stride=2)
        self.pool1 = nn.MaxPool2d(3, stride=2)
        self.rnorm1 = nn.LocalResponseNorm(64)
        
        self.conv2a = nn.Conv2d(64, 64, 1, stride=1)
        self.conv2 = nn.Conv2d(64, 192, 3, stride=1)
        self.rnorm2 = nn.LocalResponseNorm(192)
        self.pool2 = nn.MaxPool2d(3, stride=2)

        self.conv3a = nn.Conv2d(192, 192, 1, stride=1)
        self.conv3 = nn.Conv2d(192, 384, 3, stride=1)
        self.pool3 = nn.MaxPool2d(3, stride=2)

        self.conv4a = nn.Conv2d(384, 384, 1, stride=1)
        self.conv4 = nn.Conv2d(384, 256, 3, stride=1)

        self.conv5a = nn.Conv2d(256, 256, 1, stride=1)
        self.conv5 = nn.Conv2d(256, 256, 3, stride=1)

        self.conv6a = nn.Conv2d(256, 256, 1, stride=1)
        self.conv6 = nn.Conv2d(256, 256, 3, stride=1)
        self.pool4 = nn.MaxPool2d(3, stride=2)

        #todo what is concat layer?
        self.fc1 = nn.Linear(256, 128)
        self.maxout1 = nn.MaxPool2d(2)
        self.fc2 = nn.Linear(128, 128)
        self.maxout2 = nn.MaxPool2d(2)

        self.fc7128 = nn.Linear(128,128)
        
        
        

    def forward(self, x):
        x1 = self.rnorm1(self.pool1(F.relu(self.conv1(x))))
        x2 = self.pool2(self.rnorm2(F.relu(self.conv2(self.conv2a(x1)))))
        x3 = self.pool3(F.relu(self.conv3(self.conv3a(x2))))
        x4 = F.relu(self.conv4(self.conv4a(x3)))
        x5 = F.relu(self.conv5(self.conv5a(x4)))
        x6 = self.pool4(F.relu(self.conv6(self.conv6a)))
        x7 = self.maxout2(self.fc2(self.maxout1(self.fc1(x6))))
        return self.fc7128(x7)

# Train Facenet Model
training data
In all our experiments we train the CNN using Stochastic
Gradient Descent (SGD) with standard backprop [8, 11] and
AdaGrad [5]. In most experiments we start with a learning
rate of 0.05 which we lower to finalize the model. The models are initialized from random, similar to [16], and trained
on a CPU cluster for 1,000 to 2,000 hours. The decrease in
the loss (and increase in accuracy) slows down drastically
after 500h of training, but additional training can still significantly improve performance. The margin α is set to 0.2.

In [ ]:
# todo train

# Test Facenet Model
testing

In [ ]:
# todo test